In [11]:
import re
import time

from openai import OpenAI, APIError, RateLimitError
from google.colab import userdata
from collections import Counter

In [12]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("'OPENAI_API_KEY' 이름의 보안 비밀이 설정되지 않았습니다.")
client = OpenAI(api_key=OPENAI_API_KEY)

In [13]:
def extract_final_answer_simple(text: str) -> str | None:
    """텍스트 마지막 부분에서 숫자 답변 추출"""
    match = re.search(r"(?:답|Answer|답변|결과):\s*(-?\d+)", text)
    if match:
        return match.group(1)
    numbers = re.findall(r"-?\d+", text)
    return numbers[-1] if numbers else None

In [14]:
def run_self_consistency_basic(prompt: str, num_samples: int = 5) -> str:
    """기본 산술 문제에 Self-Consistency 적용"""
    print(f"\nSelf-Consistency 실행 (샘플 수: {num_samples})")

    # CoT를 유도하는 프롬프트, 이후 절에서 CoT를 자세히 설명할 예정이니
    # 해당 프롬프트는 이번에는 간략히 살펴보자
    cot_prompt = (
        f"{prompt}\n단계별로 계산하고, 최종 결과를 가장 마지막에 '답: [숫자]' 형식으로 써주세요."
    )
    # API 호출 시 재시도 로직
    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",  # 또는 사용 가능한 다른 OpenAI 모델
                messages=[{"role": "user", "content": cot_prompt}],
                temperature=0.7,
                n=num_samples,  # num_samples 만큼의 응답 생성 요청
            )
            break  # 성공 시 루프 탈출
        except (APIError, RateLimitError) as e:
            print(
                f"API 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}. 잠시 후 재시도합니다..."
            )
            if attempt == max_retries - 1:
                raise  # 마지막 시도 실패 시 오류 발생
            time.sleep(2**attempt)

    final_answers: list[str] = []
    print("\n--- 생성된 샘플 및 추출된 답 ---")

    for i, choice in enumerate(completion.choices):
        reasoning_path = choice.message.content.replace('\n', ' ')
        answer = extract_final_answer_simple(reasoning_path)
        print(f"샘플 {i + 1}: {reasoning_path[:50]}... => 추출된 답: {answer}")
        if answer:
            final_answers.append(answer)

    if not final_answers:
        return "유효한 답을 추출하지 못했습니다."

    answer_counts = Counter(final_answers)
    most_common_answer, count = answer_counts.most_common(1)[0]
    print("\n--- 집계 결과 ---")
    print(f"답변 빈도: {dict(answer_counts)}")
    print(f"최종 선택된 답: {most_common_answer} ({count}/{len(final_answers)} 회)")
    return most_common_answer

In [15]:
# 실행 및 결과 확인
prompt = "바구니에 사과가 5개 있습니다. 친구가 와서 사과의 2배만큼 오렌지를 더 주었습니다. 지금 바구니 안에는 총 몇 개의 과일이 있습니까?"
consistent_answer = run_self_consistency_basic(prompt, num_samples=5)
print(f"\n최종 일관된 답변: {consistent_answer}")


Self-Consistency 실행 (샘플 수: 5)

--- 생성된 샘플 및 추출된 답 ---
샘플 1: 1. 처음 바구니에 있는 사과의 개수는 5개입니다. 2. 친구가 주는 오렌지의 개수는 사과... => 추출된 답: 15
샘플 2: 1. 바구니에 있는 사과의 개수: 5개 2. 친구가 준 오렌지의 개수: 사과의 2배 = 5... => 추출된 답: 15
샘플 3: 1단계: 바구니에 있는 사과의 개수는 5개입니다.  2단계: 친구가 주는 오렌지의 개수를 ... => 추출된 답: 15
샘플 4: 1. 바구니에 있는 사과의 개수: 5개 2. 친구가 주는 오렌지의 개수: 사과의 2배 → ... => 추출된 답: 15
샘플 5: 1. 바구니에 있는 사과의 개수: 5개 2. 친구가 준 오렌지의 개수: 사과의 2배 → 5... => 추출된 답: 15

--- 집계 결과 ---
답변 빈도: {'15': 5}
최종 선택된 답: 15 (5/5 회)

최종 일관된 답변: 15
